# Neural Network with TensorFlow

TensorFlow has a cool interactive visualization tool of neural netowrks called [*Deep Playground*](http://playground.tensorflow.org). Let's try to config a good neural network for the four classification tasks.

## Use tf.learn
TensorFlow's high-level machine learning API (tf.learn) makes it easy to configure, train, and evaluate a variety of machine learning models. Here we are going to use DNNClassifier to build a classifier for MNIST images.

In [1]:
# Load MNIST data
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('tmp/data')

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tmp/data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tmp/data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting tmp/data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting tmp/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.p

In [2]:
X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype('int')
y_test = mnist.test.labels.astype('int')

In [3]:
# configure the neural network

# set random seed
config = tf.contrib.learn.RunConfig(tf_random_seed=42)

# define input features
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
model = tf.contrib.learn.DNNClassifier(
    hidden_units=[300, 100],
    n_classes=10,
    feature_columns=feature_cols,
    config=config
)

# make the class similar to an sklearn class
model = tf.contrib.learn.SKCompat(model)

# train the model
model.fit(X_train,
          y_train,
          batch_size=50,
          steps=4000)

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
INFO:tensorflow:Using config: {'_session_config': None, '_task_id': 0, '_save_summary_steps': 100, '_is_chief': True, '_keep_checkpoint_max': 5, '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A157B1AF28>, '_keep_checkpoint_every_n_hours': 10000, '_environment': 'local', '_num_worker_replicas': 

INFO:tensorflow:global_step/sec: 156.746
INFO:tensorflow:loss = 0.040167615, step = 3701 (0.638 sec)
INFO:tensorflow:global_step/sec: 154.487
INFO:tensorflow:loss = 0.004427076, step = 3801 (0.647 sec)
INFO:tensorflow:global_step/sec: 156.262
INFO:tensorflow:loss = 0.07642931, step = 3901 (0.640 sec)
INFO:tensorflow:Saving checkpoints for 4000 into C:\Users\Amanda\AppData\Local\Temp\tmpo11v2ptl\model.ckpt.
INFO:tensorflow:Loss for final step: 0.032751523.


SKCompat()

In [4]:
# make predictions on test set
y_pred = model.predict(X_test)

print('keys:', y_pred.keys())
print('classes for first 10 images:', y_pred['classes'][:10])

# compute accuracy
acc = sum(y_pred['classes'] == y_test) / len(y_test)
print('accuracy: {0:f}%'.format(acc))

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Amanda\AppData\Local\Temp\tmpo11v2ptl\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
keys: dict_keys(['logits', 'probabilities', 'classes'])
classes for first 10 images: [7 2 1 0 4 1 4 9 5 9]
accuracy: 0.976500%


## Use tf.estimator.DNNClassifier (For TensorFlow 1.3+)


**Note**: The textbook uses tf.learn.DNNClassifier, which is replaced by tf.estimator.DNNClassifier. The code from textbook is still working, but you will get a warning when running it.

In [5]:
# Configure the neural network

# Set input to be 28*28 
feature_cols = [tf.feature_column.numeric_column(
    'x', shape=[28, 28])]

# Create two hidden layers with 300 and 100 neurons
dnn_clf = tf.estimator.DNNClassifier(
    feature_columns=feature_cols,
    hidden_units=[300, 100],
    n_classes=10,
    model_dir='./mnist'
)

# configure training input
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x':X_train},
    y=y_train,
    num_epochs=None,
    batch_size=50,
    shuffle=True
)

# use train() method to train the model
dnn_clf.train(input_fn=train_input_fn,
              steps=4000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_task_id': 0, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_is_chief': True, '_keep_checkpoint_max': 5, '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A1591BF0B8>, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_service': None, '_model_dir': './mnist', '_evaluation_master': '', '_tf_random_seed': None, '_num_ps_replicas': 0, '_task_type': 'worker', '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./mnist\model.ckpt.
INFO:tensorflow:loss = 113.55842, step = 1
INFO:tensorflow:global_ste

In [6]:
# Define test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x':X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False
)

# Compute accuracy
evaluation = dnn_clf.evaluate(input_fn=test_input_fn)
accuracy = evaluation['accuracy']
print('Test Accuracy: {0:f}%'.format(accuracy*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-26-17:34:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-26-17:34:10
INFO:tensorflow:Saving dict for global step 4000: accuracy = 0.9729, average_loss = 0.08584396, global_step = 4000, loss = 10.866323
Test Accuracy: 97.289997%


## Use Keras

Keras is a powerful easy-to-use Python library for developing and evaluating deep neural network models. It wraps the efficient numerical computation library TensorFlow and allows you to define and train neural network models in a few lines of code.

In [10]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

C:\Users\Amanda\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
tf.reset_default_graph()
from keras import backend as K
sess = tf.Session()
K.set_session(sess)

In [13]:
img = tf.placeholder(tf.float32, shape=(None, 784))

In [14]:
x1 = Dense(300, activation='relu')(img)
x2 = Dense(100, activation='relu')(x1)
preds = Dense(10, activation='softmax')(x2)

In [15]:
labels = tf.placeholder(tf.float32, shape=(None, 10))

from keras.objectives import categorical_crossentropy
loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

In [16]:
from tensorflow.examples.tutorials.mnist import input_data
from keras.metrics import categorical_accuracy as accuracy
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
acc = accuracy(labels, preds)

init = tf.global_variables_initializer()

n_epochs = 4
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for i in range(len(mnist_data.train.images) // batch_size):
            batch = mnist_data.train.next_batch(batch_size)
            train_step.run(feed_dict={img: batch[0],
                                      labels: batch[1]})
        acc_val = acc.eval(feed_dict={img:mnist_data.test.images,
                                  labels:mnist_data.test.labels})
        print(epoch, 'accuracy:', sum(acc_val)/len(acc_val))
    

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
0 accuracy: 0.9673
1 accuracy: 0.9688
2 accuracy: 0.9738
3 accuracy: 0.9682


## Using plain TensorFlow

In [17]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [18]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32,
                   shape=(None, n_inputs),
                   name='x')
y = tf.placeholder(tf.int64,
                   shape=(None),
                   name='y')

In [19]:
def neuron_layer(X, n_neurons, name, activation):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal(
            (n_inputs, n_neurons),
            stddev=stddev
        )
        W = tf.Variable(init, name='weights')
        b = tf.Variable(tf.zeros([n_neurons]),
                        name='bias')
        Z = tf.matmul(X, W) + b
        return activation(Z)

In [20]:
with tf.name_scope('dnn'):
    hidden1 = neuron_layer(
        X,
        n_hidden1,
        name='hidden1',
        activation=tf.nn.relu
    )
    hidden2 = neuron_layer(
        hidden1,
        n_hidden2,
        name='hidden2',
        activation=tf.nn.relu
    )
    logits = neuron_layer(
        hidden2,
        n_outputs,
        name='outputs',
        activation=tf.identity
    )

In [21]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y,
        logits=logits
    )
    loss = tf.reduce_mean(xentropy,
                          name='loss')

In [22]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(
        learning_rate
    )
    training_op = optimizer.minimize(loss)

In [23]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(
        tf.cast(correct, tf.float32)
    )

In [24]:
init = tf.global_variables_initializer()

In [25]:
n_epochs = 40
batch_size = 50
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op,
                     feed_dict={X: X_batch,
                                y: y_batch})
        acc = accuracy.eval(
            feed_dict={X: X_test,
                       y: y_test}
        )
        print(epoch, 'Accuracy:', acc)

0 Accuracy: 0.9113
1 Accuracy: 0.9279
2 Accuracy: 0.9372
3 Accuracy: 0.9442
4 Accuracy: 0.9469
5 Accuracy: 0.9541
6 Accuracy: 0.958
7 Accuracy: 0.9578
8 Accuracy: 0.9618
9 Accuracy: 0.9611
10 Accuracy: 0.9632
11 Accuracy: 0.9643
12 Accuracy: 0.9662
13 Accuracy: 0.9674
14 Accuracy: 0.9685
15 Accuracy: 0.9695
16 Accuracy: 0.9692
17 Accuracy: 0.9697
18 Accuracy: 0.9698
19 Accuracy: 0.9707
20 Accuracy: 0.9711
21 Accuracy: 0.9713
22 Accuracy: 0.9731
23 Accuracy: 0.9722
24 Accuracy: 0.9734
25 Accuracy: 0.9749
26 Accuracy: 0.975
27 Accuracy: 0.9754
28 Accuracy: 0.9748
29 Accuracy: 0.9753
30 Accuracy: 0.9757
31 Accuracy: 0.9755
32 Accuracy: 0.975
33 Accuracy: 0.9758
34 Accuracy: 0.9761
35 Accuracy: 0.9769
36 Accuracy: 0.9772
37 Accuracy: 0.976
38 Accuracy: 0.9766
39 Accuracy: 0.9769
